In [23]:
import os
import time
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import random

from copy import deepcopy
from collections import namedtuple

from yaml import load, dump
try:
    from yaml import CLoader as Loader, CDumper as Dumper
except ImportError:
    from yaml import Loader, Dumper

import pydrake
from pydrake.all import (
    AngleAxis,
    CommonSolverOption,
    MathematicalProgram,
    MakeSolver,
    RigidTransform,
    RollPitchYaw,
    RotationMatrix,
    GurobiSolver,
    Solve,
    SolverOptions,
    VPolytope,
    MixedIntegerRotationConstraintGenerator,
    IntervalBinning
)

def solve(N):
    print("Starting %d" % N)
    prog = MathematicalProgram()
    for k in range(N):
        R = prog.NewContinuousVariables(3, 3, "R")
        mip_rot_gen = MixedIntegerRotationConstraintGenerator(
            approach = MixedIntegerRotationConstraintGenerator.Approach.kBilinearMcCormick,
            num_intervals_per_half_axis=2,
            interval_binning = IntervalBinning.kLogarithmic
        )
        mip_rot_gen.AddToProgram(R, prog)
        prog.AddCost(np.sum(R))
    start_time = time.time()
    result = Solve(prog)
    assert result.is_success()
    solve_time = time.time() - start_time
    print("Solved %d in %f" % (N, solve_time))

solve(5)

Starting 5
Solved 5 in 0.496419
